In [1]:
import sys
import os
import importlib
import shutil
import numpy as np
import threading
# import keras
import tqdm
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

import selectivesearch
importlib.reload(selectivesearch)
from selectivesearch import get_selective_search_regions

import PIL

import utils
importlib.reload(utils)
from utils import *

import pascal_voc_reader

import bbox_transform
importlib.reload(bbox_transform)
from bbox_transform import *

import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.init as weight_init
import torch.nn.functional as F

import rcnn_utils
importlib.reload(rcnn_utils)
from rcnn_utils import *

import data_utils
importlib.reload(data_utils)
from data_utils import *

from IPython.core.debugger import Tracer
from IPython.display import clear_output

%matplotlib inline

from roi_pooling.modules.roi_pool import RoIPool as RoIPool_GPU

## Load Data

In [ ]:
image_size = (500, 500)
dataset = RCNN_All_Data(image_size)
trn_dataset = RCNN_Set(dataset, 'train')
val_dataset = RCNN_Set(dataset, 'valid')

## Fast RCNN after conv layers

In [ ]:
try:
    del fast_rcnn_model
except NameError:
    pass

fast_rcnn_model = Fast_RCNN_model(dropout_p=0.1).cuda()
fast_rcnn_weights_init(fast_rcnn_model)

In [ ]:
# test
batch = get_batch(use_features=True,
                  images_per_batch=1,
                  roi_batch_size=64,
                  loop_over=False)
cnn_features, rois, targets = batch

cnn_features_var = np_to_var(cnn_features.astype(np.float32))
rois_var = np_to_var(rois.astype(np.int32))

out = fast_rcnn_model.forward(cnn_features_var, rois_var)

out[0].size(), out[1].size()

## Train loop

In [9]:
def train(net, loss_criterion, optimizer, epoch=1, print_every=10):
    net.train()
    total_loss = 0
    total_acc = 0
    images_per_batch = 1
    roi_batch_size = 64
    n_batches = (len(train_samples) // images_per_batch)
    for epoch_index in range(epoch):
        t = tqdm.tqdm_notebook(total=n_batches)
        batch_index = 1
        while True:
            batch = get_batch(images_per_batch=images_per_batch,
                              roi_batch_size=roi_batch_size,
                              use_features=True, loop_over=False)
            images, rois, targets = batch
            if len(images) == 0 or len(rois) == 0 or len(targets) == 0:
                reset_index('train')
                break

            images_var = np_to_var(images.astype(np.float32))
            # images_var = Variable(images.cuda())
            rois_var = np_to_var(rois.astype(np.int32))
            targets_var = np_to_var(targets)

            # Clear Gradients
            optimizer.zero_grad()

            # Forward Pass
            output = net(images_var, rois_var)

            loss = loss_criterion(output, targets_var)

            # Backprop
            loss.backward()
            optimizer.step()

            # preds = get_predictions(output)
            accuracy = 0  # = get_accuracy(preds, targets.data.cpu().numpy())

            total_loss += loss.data[0]
            current_mean_loss = total_loss / \
                (batch_index + n_batches * epoch_index)
            if (batch_index + 1) % print_every == 0:
                clear_output()
                print(current_mean_loss, end=' ')

            total_acc += accuracy
            batch_index += 1
            t.update()
        t.close()
        mean_loss = total_loss / n_batches
        mean_acc = total_acc / n_batches
    return mean_loss, mean_acc


def get_predictions(model_output):
    # Flatten and Get ArgMax to compute accuracy
    #     val,idx = torch.max(model_output, dim=1)
    #     return idx.data.cpu().view(-1).numpy()
    pass


def get_accuracy(preds, targets):
    pass

#### Run the training

In [ ]:
saved_path = 'intermediate/voc/weights-regression-300.pth'
load_weights(fast_rcnn_model, saved_path)

In [ ]:
reset_index('train')
learning_rate = 1e-3
optimizer = torch.optim.Adam(fast_rcnn_model.parameters(), lr=learning_rate)
train(fast_rcnn_model, loss_criterion, optimizer, epoch=100)

In [ ]:
reset_index('train')
learning_rate = 1e-4
optimizer = torch.optim.Adam(fast_rcnn_model.parameters(), lr=learning_rate)
train(fast_rcnn_model, loss_criterion, optimizer, epoch=100)

In [ ]:
reset_index('train')
learning_rate = 1e-5
optimizer = torch.optim.Adam(fast_rcnn_model.parameters(), lr=learning_rate)
train(fast_rcnn_model, loss_criterion, optimizer, epoch=100)

In [ ]:
reset_index('train')
learning_rate = 1e-6
optimizer = torch.optim.Adam(fast_rcnn_model.parameters(), lr=learning_rate)
train(fast_rcnn_model, loss_criterion, optimizer, epoch=100)

In [ ]:
saved_path = save_weights(fast_rcnn_model, 'intermediate/voc', 300)
saved_path

#### Test the net

In [ ]:
batch = get_batch('valid',
                  images_per_batch=1,
                  roi_batch_size=64,
                  use_features=True, loop_over=False)
images, rois, targets = batch

images_var = np_to_var(images.astype(np.float32))
# images_var = Variable(images.cuda())

rois_var = np_to_var(rois.astype(np.int32))
targets_var = np_to_var(targets)

out = fast_rcnn_model(images_var, rois_var)

probas, indexes = torch.max(out[0], 1)
indexes_np = np.squeeze(indexes.data.cpu().numpy())
non_bg_rois_indexes = (indexes_np > 0)
'Total Detected: ', np.sum(non_bg_rois_indexes)

non_bg_rois = rois_var.data.cpu().numpy()[non_bg_rois_indexes]

detected_roi = np.append(non_bg_rois[:, 1:],
                         indexes_np[non_bg_rois_indexes][:, None],
                         axis=1)

detected_roi.shape

image_arr = valid_samples[valid_index[0] - 1]
image = PIL.Image.fromarray(image_arr.astype('uint8'))

for class_id in detected_roi[:, 4]:
    print(class_id_to_name[class_id])

display_image_regions(image, detected_roi)